In [36]:
#check BIOGEME version
import biogeme.version as ver
print("Biogeme version:", ver.getVersion())

#Import necessary libraries
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models
from biogeme.expressions import Beta, Variable
from tqdm.autonotebook import tqdm

import numpy as np
import math as math
import scipy.stats as stats

Biogeme version: 3.3.0


/var/folders/g0/rncnm3l57mvd8n3r12bv0_br0000gn/T/ipykernel_2916/667974273.py:3: DeprecationWarning: getVersion is deprecated; use get_version instead.
  print("Biogeme version:", ver.getVersion())


In [37]:
# Load the database
df = pd.read_csv("SwissRouteChoiceData.csv", sep=',') # Check the path to your CSV file
# Check missing values
print(df.isnull().sum())
df

ID                  0
choice              0
tt1                 0
tc1                 0
hw1                 0
ch1                 0
tt2                 0
tc2                 0
hw2                 0
ch2                 0
hh_inc_abs          0
car_availability    0
commute             0
shopping            0
business            0
leisure             0
dtype: int64


,ID,choice,tt1,tc1,hw1,ch1,tt2,tc2,hw2,ch2,hh_inc_abs,car_availability,commute,shopping,business,leisure
0,2439,2,58,7,30,1,50,8,30,0,50000,1,1,0,0,0
1,2439,1,30,8,60,0,41,7,15,2,50000,1,1,0,0,0
2,2439,1,41,7,30,0,34,8,15,2,50000,1,1,0,0,0
3,2439,1,44,10,60,1,52,9,60,2,50000,1,1,0,0,0
4,2439,2,43,9,60,0,34,10,30,0,50000,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3487,84525,1,22,4,15,0,24,3,15,0,70000,0,1,0,0,0
3488,84525,1,22,3,15,0,20,4,30,1,70000,0,1,0,0,0
3489,84525,1,20,3,15,2,16,4,15,2,70000,0,1,0,0,0
3490,84525,2,20,4,60,2,15,5,30,2,70000,0,1,0,0,0


In [38]:
#Pre-process the data: scale down the income variable
df['hh_inc_abs'] = df['hh_inc_abs'] / 1000
df

,ID,choice,tt1,tc1,hw1,ch1,tt2,tc2,hw2,ch2,hh_inc_abs,car_availability,commute,shopping,business,leisure
0,2439,2,58,7,30,1,50,8,30,0,50.0,1,1,0,0,0
1,2439,1,30,8,60,0,41,7,15,2,50.0,1,1,0,0,0
2,2439,1,41,7,30,0,34,8,15,2,50.0,1,1,0,0,0
3,2439,1,44,10,60,1,52,9,60,2,50.0,1,1,0,0,0
4,2439,2,43,9,60,0,34,10,30,0,50.0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3487,84525,1,22,4,15,0,24,3,15,0,70.0,0,1,0,0,0
3488,84525,1,22,3,15,0,20,4,30,1,70.0,0,1,0,0,0
3489,84525,1,20,3,15,2,16,4,15,2,70.0,0,1,0,0,0
3490,84525,2,20,4,60,2,15,5,30,2,70.0,0,1,0,0,0


In [39]:
# Define the BIOGEME-specific database
database = db.Database("SwissChoice", df)
#print the BIOGEME-specific database
database._df

,ID,choice,tt1,tc1,hw1,ch1,tt2,tc2,hw2,ch2,hh_inc_abs,car_availability,commute,shopping,business,leisure
0,2439.0,2.0,58.0,7.0,30.0,1.0,50.0,8.0,30.0,0.0,50.0,1.0,1.0,0.0,0.0,0.0
1,2439.0,1.0,30.0,8.0,60.0,0.0,41.0,7.0,15.0,2.0,50.0,1.0,1.0,0.0,0.0,0.0
2,2439.0,1.0,41.0,7.0,30.0,0.0,34.0,8.0,15.0,2.0,50.0,1.0,1.0,0.0,0.0,0.0
3,2439.0,1.0,44.0,10.0,60.0,1.0,52.0,9.0,60.0,2.0,50.0,1.0,1.0,0.0,0.0,0.0
4,2439.0,2.0,43.0,9.0,60.0,0.0,34.0,10.0,30.0,0.0,50.0,1.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3487,84525.0,1.0,22.0,4.0,15.0,0.0,24.0,3.0,15.0,0.0,70.0,0.0,1.0,0.0,0.0,0.0
3488,84525.0,1.0,22.0,3.0,15.0,0.0,20.0,4.0,30.0,1.0,70.0,0.0,1.0,0.0,0.0,0.0
3489,84525.0,1.0,20.0,3.0,15.0,2.0,16.0,4.0,15.0,2.0,70.0,0.0,1.0,0.0,0.0,0.0
3490,84525.0,2.0,20.0,4.0,60.0,2.0,15.0,5.0,30.0,2.0,70.0,0.0,1.0,0.0,0.0,0.0


In [40]:
"""
Use the loop below to get all of columns names.
for col in df.columns:
    print(f'{col} = Variable("{col}")')

1. The code will generate the Variable for each column.    
2. Copy the output and paste it below to define the variables.
"""
for col in df.columns:
    print(f'{col} = Variable("{col}")')

ID = Variable("ID")
choice = Variable("choice")
tt1 = Variable("tt1")
tc1 = Variable("tc1")
hw1 = Variable("hw1")
ch1 = Variable("ch1")
tt2 = Variable("tt2")
tc2 = Variable("tc2")
hw2 = Variable("hw2")
ch2 = Variable("ch2")
hh_inc_abs = Variable("hh_inc_abs")
car_availability = Variable("car_availability")
commute = Variable("commute")
shopping = Variable("shopping")
business = Variable("business")
leisure = Variable("leisure")


In [41]:
#Define the variables
ID = Variable("ID")
choice = Variable("choice")
tt1 = Variable("tt1")
tc1 = Variable("tc1")
hw1 = Variable("hw1")
ch1 = Variable("ch1")
tt2 = Variable("tt2")
tc2 = Variable("tc2")
hw2 = Variable("hw2")
ch2 = Variable("ch2")
hh_inc_abs = Variable("hh_inc_abs")
car_availability = Variable("car_availability")
commute = Variable("commute")
shopping = Variable("shopping")
business = Variable("business")
leisure = Variable("leisure")
# REMEMBER to define new variables if you have modified the dataset!

## MODEL 1

In [42]:
# Define beta values to be estimated
## Beta('name', initial value, lower bound, upper bound, reference)
## Reference is used to set the reference alternative in a logit model (1 = yes, 0 = no)
asc_route1 = Beta('asc_route1', 0, None, None, 0)
asc_route2 = Beta('asc_route2', 0, None, None, 1) # reference alternative
b_cost = Beta('b_cost', 0, None, None, 0)
b_time = Beta('b_time', 0, None, None, 0)

# Utility functions
V_route1 = asc_route1 + b_cost * tc1 + b_time * tt1
V_route2 = asc_route2 + b_cost * tc2 + b_time * tt2

# Dictionary defining alternatives mapping
V = {1: V_route1, 2: V_route2}

# Dictionary defining availability mapping
av = {1: 1, 2: 1} # Both alternatives are ALWAYS available

# Define the choice model
# loglogit(Alternatives mapping, Availability mapping, Choice variable)
logprob= models.loglogit(V, av, choice)

# Estimate Model
#the_biogeme = bio.BIOGEME(database, logprob)
the_biogeme = bio.BIOGEME(database, logprob)
the_biogeme.model_name = 'Swiss1' # Set the model name

#Calculate null Loglikelihood
the_biogeme.calculate_null_loglikelihood(av)

# Save the estimation results
Swiss1 = the_biogeme.estimate()

#Print the results
print(Swiss1.short_summary())
Swiss1.get_estimated_parameters()

Results for model Swiss1
Nbr of parameters:		3
Sample size:			3492
Excluded data:			0
Null log likelihood:		-2420.47
Final log likelihood:		-2349.37
Likelihood ratio test (null):		142.1992
Rho square (null):			0.0294
Rho bar square (null):			0.0281
Akaike Information Criterion:	4704.741
Bayesian Information Criterion:	4723.215



/var/folders/g0/rncnm3l57mvd8n3r12bv0_br0000gn/T/ipykernel_2916/1509738149.py:36: DeprecationWarning: get_estimated_parameters is deprecated. Use get_pandas_estimated_parameters(estimation_results=my_results) instead
  Swiss1.get_estimated_parameters()


,Name,Value,Robust std err.,Robust t-stat.,Robust p-value
0,asc_route1,-0.011959,0.034466,-0.346987,7.286009e-01
1,b_cost,-0.091266,0.013516,-6.752555,1.452638e-11
2,b_time,-0.037280,0.003915,-9.522309,0.000000e+00


## MODEL 2

In [43]:
# Define beta values to be estimated
## Beta('name', initial value, lower bound, upper bound, reference)
## Reference is used to set the reference alternative in a logit model (1 = yes, 0 = no)
asc_route1 = Beta('asc_route1', 0, None, None, 0)
asc_route2 = Beta('asc_route2', 0, None, None, 1) # reference alternative
b_cost = Beta('b_cost', 0, None, None, 0)
b_time = Beta('b_time', 0, None, None, 0)
b_hw = Beta('b_hw', 0, None, None, 0)
b_income = Beta('b_income', 0, None, None, 0)
b_cost_inc = Beta('b_cost_inc', 0, None, None, 0)

# Utility functions
V_route1 = (asc_route1 + 
            b_cost * tc1 + 
            b_time * tt1 +
            b_hw * hw1 +
            b_income * hh_inc_abs +
            b_cost_inc * (tc1 / hh_inc_abs)
            )
V_route2 = (asc_route2 + 
            b_cost * tc2 + 
            b_time * tt2 +
            b_hw * hw2 +
            b_cost_inc * (tc2 / hh_inc_abs)
            )

# Dictionary defining alternatives mapping
V = {1: V_route1, 2: V_route2}

# Dictionary defining availability mapping
av = {1: 1, 2: 1} # Both alternatives are ALWAYS available

# Define the choice model
# loglogit(Alternatives mapping, Availability mapping, Choice variable)
logprob= models.loglogit(V, av, choice)

# Estimate Model
#the_biogeme = bio.BIOGEME(database, logprob)
the_biogeme = bio.BIOGEME(database, logprob)
the_biogeme.model_name = 'Swiss2' # Set the model name

#Calculate null Loglikelihood
the_biogeme.calculate_null_loglikelihood(av)

# Save the estimation results
Swiss2 = the_biogeme.estimate()

#Print the results
print(Swiss2.short_summary())
Swiss2.get_estimated_parameters()

Results for model Swiss2
Nbr of parameters:		6
Sample size:			3492
Excluded data:			0
Null log likelihood:		-2420.47
Final log likelihood:		-2148.186
Likelihood ratio test (null):		544.5677
Rho square (null):			0.112
Rho bar square (null):			0.11
Akaike Information Criterion:	4308.372
Bayesian Information Criterion:	4345.322



/var/folders/g0/rncnm3l57mvd8n3r12bv0_br0000gn/T/ipykernel_2916/2857566267.py:50: DeprecationWarning: get_estimated_parameters is deprecated. Use get_pandas_estimated_parameters(estimation_results=my_results) instead
  Swiss2.get_estimated_parameters()


,Name,Value,Robust std err.,Robust t-stat.,Robust p-value
0,asc_route1,0.097156,0.074194,1.309485,1.903701e-01
1,b_cost,-0.090785,0.015099,-6.012458,1.827308e-09
2,b_time,-0.043194,0.004278,-10.095896,0.000000e+00
3,b_hw,-0.027330,0.001472,-18.562990,0.000000e+00
4,b_income,-0.001350,0.000824,-1.639065,1.011997e-01
5,b_cost_inc,-0.562723,0.296029,-1.900906,5.731433e-02


## MODEL 3

In [44]:
# Define beta values to be estimated
## Beta('name', initial value, lower bound, upper bound, reference)
## Reference is used to set the reference alternative in a logit model (1 = yes, 0 = no)
asc_route1 = Beta('asc_route1', 0, None, None, 0)
asc_route2 = Beta('asc_route2', 0, None, None, 1) # reference alternative
b_cost = Beta('b_cost', 0, None, None, 0)
b_time = Beta('b_time', 0, None, None, 0)
b_hw = Beta('b_hw', 0, None, None, 0)
b_income = Beta('b_income', 0, None, None, 0)
b_cost_inc = Beta('b_cost_inc', 0, None, None, 0)
b_shopping = Beta('b_shopping', 0, None, None, 0)
b_leisure = Beta('b_leisure', 0, None, None, 0)

# Utility functions
V_route1 = (asc_route1 + 
            b_cost * tc1 + 
            b_time * tt1 +
            b_hw * hw1 +
            b_income * hh_inc_abs +
            b_cost_inc * (tc1 / hh_inc_abs) +
            b_shopping * shopping +
            b_leisure * leisure
            
            )
V_route2 = (asc_route2 + 
            b_cost * tc2 + 
            b_time * tt2 +
            b_hw * hw2 +
            b_cost_inc * (tc2 / hh_inc_abs)
            )

# Dictionary defining alternatives mapping
V = {1: V_route1, 2: V_route2}

# Dictionary defining availability mapping
av = {1: 1, 2: 1} # Both alternatives are ALWAYS available

# Define the choice model
# loglogit(Alternatives mapping, Availability mapping, Choice variable)
logprob= models.loglogit(V, av, choice)

# Estimate Model
#the_biogeme = bio.BIOGEME(database, logprob)
the_biogeme = bio.BIOGEME(database, logprob)
the_biogeme.model_name = 'Swiss3' # Set the model name

#Calculate null Loglikelihood
the_biogeme.calculate_null_loglikelihood(av)

# Save the estimation results
Swiss3 = the_biogeme.estimate()

#Print the results
print(Swiss3.short_summary())
Swiss3.get_estimated_parameters()

Results for model Swiss3
Nbr of parameters:		8
Sample size:			3492
Excluded data:			0
Null log likelihood:		-2420.47
Final log likelihood:		-2147.985
Likelihood ratio test (null):		544.9696
Rho square (null):			0.113
Rho bar square (null):			0.109
Akaike Information Criterion:	4311.97
Bayesian Information Criterion:	4361.236



/var/folders/g0/rncnm3l57mvd8n3r12bv0_br0000gn/T/ipykernel_2916/1242675986.py:55: DeprecationWarning: get_estimated_parameters is deprecated. Use get_pandas_estimated_parameters(estimation_results=my_results) instead
  Swiss3.get_estimated_parameters()


,Name,Value,Robust std err.,Robust t-stat.,Robust p-value
0,asc_route1,0.108103,0.093993,1.150115,2.500965e-01
1,b_cost,-0.090726,0.015100,-6.008154,1.876470e-09
2,b_time,-0.043185,0.004277,-10.097029,0.000000e+00
3,b_hw,-0.027316,0.001472,-18.552150,0.000000e+00
4,b_income,-0.001388,0.000839,-1.653604,9.820805e-02
5,b_cost_inc,-0.561004,0.296304,-1.893338,5.831292e-02
6,b_shopping,0.058379,0.135002,0.432430,6.654288e-01
7,b_leisure,-0.024447,0.078610,-0.310985,7.558117e-01


## MODEL 4

In [45]:
# Define beta values to be estimated
## Beta('name', initial value, lower bound, upper bound, reference)
## Reference is used to set the reference alternative in a logit model (1 = yes, 0 = no)
asc_route1 = Beta('asc_route1', 0, None, None, 0)
asc_route2 = Beta('asc_route2', 0, None, None, 1) # reference alternative
b_cost = Beta('b_cost', 0, None, None, 0)
b_time = Beta('b_time', 0, None, None, 0)
b_hw = Beta('b_hw', 0, None, None, 0)
b_income = Beta('b_income', 0, None, None, 0)
b_cost_inc = Beta('b_cost_inc', 0, None, None, 0)
b_shopping = Beta('b_shopping', 0, None, None, 0)
b_leisure = Beta('b_leisure', 0, None, None, 0)

# Utility functions
V_route1 = (asc_route1 + 
            b_cost * tc1 + 
            b_time * tt1 +
            b_hw * hw1 +
            b_income * hh_inc_abs +
            b_cost_inc * (tc1 / hh_inc_abs) +
            b_shopping * shopping +
            b_leisure * leisure
            
            )
V_route2 = (asc_route2 + 
            b_cost * tc2 + 
            b_time * tt2 +
            b_hw * hw2 +
            b_cost_inc * (tc2 / hh_inc_abs)
            )

# Dictionary defining alternatives mapping
V = {1: V_route1, 2: V_route2}

# Dictionary defining availability mapping
av = {1: 1, 2: 1} # Both alternatives are ALWAYS available

# Define the choice model
# loglogit(Alternatives mapping, Availability mapping, Choice variable)
logprob= models.loglogit(V, av, choice)

# Estimate Model
#the_biogeme = bio.BIOGEME(database, logprob)
the_biogeme = bio.BIOGEME(database, logprob)
the_biogeme.model_name = 'Swiss4' # Set the model name

#Calculate null Loglikelihood
the_biogeme.calculate_null_loglikelihood(av)

# Save the estimation results
Swiss4 = the_biogeme.estimate()

#Print the results
print(Swiss4.short_summary())
Swiss4.get_estimated_parameters()

Results for model Swiss4
Nbr of parameters:		8
Sample size:			3492
Excluded data:			0
Null log likelihood:		-2420.47
Final log likelihood:		-2147.985
Likelihood ratio test (null):		544.9696
Rho square (null):			0.113
Rho bar square (null):			0.109
Akaike Information Criterion:	4311.97
Bayesian Information Criterion:	4361.236



/var/folders/g0/rncnm3l57mvd8n3r12bv0_br0000gn/T/ipykernel_2916/976901347.py:55: DeprecationWarning: get_estimated_parameters is deprecated. Use get_pandas_estimated_parameters(estimation_results=my_results) instead
  Swiss4.get_estimated_parameters()


,Name,Value,Robust std err.,Robust t-stat.,Robust p-value
0,asc_route1,0.108103,0.093993,1.150115,2.500965e-01
1,b_cost,-0.090726,0.015100,-6.008154,1.876470e-09
2,b_time,-0.043185,0.004277,-10.097029,0.000000e+00
3,b_hw,-0.027316,0.001472,-18.552150,0.000000e+00
4,b_income,-0.001388,0.000839,-1.653604,9.820805e-02
5,b_cost_inc,-0.561004,0.296304,-1.893338,5.831292e-02
6,b_shopping,0.058379,0.135002,0.432430,6.654288e-01
7,b_leisure,-0.024447,0.078610,-0.310985,7.558117e-01


## Value of Travel Time

$$\text{VoTT} = \frac{\partial V/\partial\text{TT}}{\partial V/\partial\text{TC}}=\frac{\beta_T}{\beta_C}$$

In [46]:
#get beta time values
beta_time = Swiss4.get_beta_values().get('b_time')
beta_cost = Swiss4.get_beta_values().get('b_cost')

VoTT = beta_time / beta_cost
print(VoTT)

0.4759969640956304


## LR Test

$$ \text{LR Test} = -2\left(\mathcal{L}\left(\widehat{\beta_{\text{Re}}}\right)-\mathcal{L}\left(\widehat{\beta_{\text{Un}}}\right)\right) \sim \chi^2_{\left(\kappa_{\text{Un}}-\kappa_{\text{Re}}\right)} $$

In [ ]:
# Get the log likelihood of each model
LL_Swiss1 = Swiss1.final_loglikelihood
LL_Swiss2 = Swiss2.final_loglikelihood
LL_Swiss3 = Swiss3.final_loglikelihood
LL_Swiss4 = Swiss4.final_loglikelihood

import scipy.stats as stats

#Compute the LR Test value
LR = -2 * (LL_Swiss1 - LL_Swiss2)

#Determine the degrees of freedom
q = Swiss2.number_of_parameters - Swiss1.number_of_parameters
#Compute the critical value and p-value
critvalue = stats.chi2.ppf(0.95, q)
p_value = 1 - stats.chi2.cdf(LR, q)

print(f"LR statistic: {LR:.3f}")
print(f"Degrees of freedom: {q}")
print(f"Critical value at 5%: {critvalue:.3f}")
print(f"p-value: {p_value:.4f}")

if LR > critvalue:
    print("Reject the null hypothesis: The more complex model (Swiss2) provides a significantly better fit.")
else:
    print("Fail to reject the null hypothesis: The simpler model (Swiss1) is sufficient.")

LR statistic: 402.368
Degrees of freedom: 3
Critical value at 5%: 7.815
p-value: 0.0000
Reject the null hypothesis: The more complex model (Swiss2) provides a significantly better fit.


We can make the LR-Test as a function so we can re use them

In [48]:
def LR_test(model_1, model_2):
    #retrieve log-likelihoods
    
    if model_1.number_of_parameters == model_2.number_of_parameters: 
        raise ValueError("Both models have the same number of parameters. LR test is not applicable.")
    model_simple = model_1 if model_1.number_of_parameters < model_2.number_of_parameters else model_2
    model_complex = model_2 if model_1.number_of_parameters < model_2.number_of_parameters else model_1
    
    LL_simple = model_simple.final_loglikelihood
    LL_complex = model_complex.final_loglikelihood
    #compute LR statistic
    LR = -2 * (LL_simple - LL_complex)
    #degrees of freedom
    q = model_complex.number_of_parameters - model_simple.number_of_parameters
    #compute critical value and p-value
    critvalue = float(stats.chi2.ppf(0.95, q))
    p_value = float(1 - stats.chi2.cdf(LR, q))
    result = lambda x: "Reject the null hypothesis: The" + model_complex.model_name + " provides a significantly better fit." if LR > critvalue else "Fail to reject the null hypothesis: The " + model_simple.model_name + " is sufficient."
    return {
        "LR_statistic": LR,
        "Degrees_of_freedom": q,
        "Critical_value_5%": critvalue,
        "p_value": p_value,
        "Reject_null": result(LR)
    }


## Updated Model 3 without insignificant attributes/characteristics

Attributes with t-ratio < 1.96 or t-ratio > −1.96 are insignificant. Based on Model 3 result, the insignificant attributes are:
1. income 
2. leisure
3. shopping

In [49]:
# Define beta values to be estimated
## Beta('name', initial value, lower bound, upper bound, reference)
## Reference is used to set the reference alternative in a logit model (1 = yes, 0 = no)
asc_route1 = Beta('asc_route1', 0, None, None, 0)
asc_route2 = Beta('asc_route2', 0, None, None, 1) # reference alternative
b_cost = Beta('b_cost', 0, None, None, 0)
b_time = Beta('b_time', 0, None, None, 0)
b_hw = Beta('b_hw', 0, None, None, 0)
#b_income = Beta('b_income', 0, None, None, 0)
b_cost_inc = Beta('b_cost_inc', 0, None, None, 0)
#b_shopping = Beta('b_shopping', 0, None, None, 0)
#b_leisure = Beta('b_leisure', 0, None, None, 0)

# Utility functions
V_route1 = (asc_route1 + 
            b_cost * tc1 + 
            b_time * tt1 +
            b_hw * hw1 +
            #b_income * hh_inc_abs +
            b_cost_inc * (tc1 / hh_inc_abs)
            #b_shopping * shopping +
            #b_leisure * leisure            
            )
V_route2 = (asc_route2 + 
            b_cost * tc2 + 
            b_time * tt2 +
            b_hw * hw2 +
            b_cost_inc * (tc2 / hh_inc_abs)
            )

# Dictionary defining alternatives mapping
V = {1: V_route1, 2: V_route2}

# Dictionary defining availability mapping
av = {1: 1, 2: 1} # Both alternatives are ALWAYS available

# Define the choice model
# loglogit(Alternatives mapping, Availability mapping, Choice variable)
logprob= models.loglogit(V, av, choice)

# Estimate Model
#the_biogeme = bio.BIOGEME(database, logprob)
the_biogeme = bio.BIOGEME(database, logprob)
the_biogeme.model_name = 'Swiss3.1' # Set the model name

#Calculate null Loglikelihood
the_biogeme.calculate_null_loglikelihood(av)

# Save the estimation results
Swiss3_1 = the_biogeme.estimate()

#Print the results
print(Swiss3_1.short_summary())
Swiss3_1.get_estimated_parameters()

Results for model Swiss3.1
Nbr of parameters:		5
Sample size:			3492
Excluded data:			0
Null log likelihood:		-2420.47
Final log likelihood:		-2149.524
Likelihood ratio test (null):		541.8911
Rho square (null):			0.112
Rho bar square (null):			0.11
Akaike Information Criterion:	4309.049
Bayesian Information Criterion:	4339.84



/var/folders/g0/rncnm3l57mvd8n3r12bv0_br0000gn/T/ipykernel_2916/690849823.py:54: DeprecationWarning: get_estimated_parameters is deprecated. Use get_pandas_estimated_parameters(estimation_results=my_results) instead
  Swiss3_1.get_estimated_parameters()


,Name,Value,Robust std err.,Robust t-stat.,Robust p-value
0,asc_route1,-0.006264,0.036589,-0.171192,8.640724e-01
1,b_cost,-0.090461,0.015082,-5.997761,2.000568e-09
2,b_time,-0.043035,0.004273,-10.071062,0.000000e+00
3,b_hw,-0.027314,0.001470,-18.584637,0.000000e+00
4,b_cost_inc,-0.545914,0.296798,-1.839345,6.586445e-02


In [50]:
# Perform the LR test between Swiss3 and Swiss3_1
LR_test(Swiss3, Swiss3_1)

{'LR_statistic': 3.0785207323333452,
 'Degrees_of_freedom': 3,
 'Critical_value_5%': 7.8147279032511765,
 'p_value': 0.3796765528291355,
 'Reject_null': 'Fail to reject the null hypothesis: The Swiss3.1 is sufficient.'}

## MODEL 5

In [51]:

# Update the dataset with log-transformed cost variables
df['log_tc1'] = np.log(df['tc1'])
df['log_tc2'] = np.log(df['tc2'])
# Update the BIOGEME-specific database
database = db.Database("SwissChoice", df)
#update Variables
log_tc1 = Variable("log_tc1")
log_tc2 = Variable("log_tc2")

In [52]:
# Define beta values to be estimated
## Beta('name', initial value, lower bound, upper bound, reference)
## Reference is used to set the reference alternative in a logit model (1 = yes, 0 = no)
asc_route1 = Beta('asc_route1', 0, None, None, 0)
asc_route2 = Beta('asc_route2', 0, None, None, 1) # reference alternative
b_cost = Beta('b_cost', 0, None, None, 0)

# Utility functions
V_route1 = asc_route1 + b_cost * log_tc1 
V_route2 = asc_route2 + b_cost * log_tc2 

# Dictionary defining alternatives mapping
V = {1: V_route1, 2: V_route2}

# Dictionary defining availability mapping
av = {1: 1, 2: 1} # Both alternatives are ALWAYS available

# Define the choice model
# loglogit(Alternatives mapping, Availability mapping, Choice variable)
logprob= models.loglogit(V, av, choice)

# Estimate Model
#the_biogeme = bio.BIOGEME(database, logprob)
the_biogeme = bio.BIOGEME(database, logprob)
the_biogeme.model_name = 'Swiss5' # Set the model name

#Calculate null Loglikelihood
the_biogeme.calculate_null_loglikelihood(av)

# Save the estimation results
Swiss5 = the_biogeme.estimate()

#Print the results
print(Swiss5.short_summary())
Swiss5.get_estimated_parameters()

Results for model Swiss5
Nbr of parameters:		2
Sample size:			3492
Excluded data:			0
Null log likelihood:		-2420.47
Final log likelihood:		-2411.609
Likelihood ratio test (null):		17.72225
Rho square (null):			0.00366
Rho bar square (null):			0.00283
Akaike Information Criterion:	4827.218
Bayesian Information Criterion:	4839.534



/var/folders/g0/rncnm3l57mvd8n3r12bv0_br0000gn/T/ipykernel_2916/3042986530.py:35: DeprecationWarning: get_estimated_parameters is deprecated. Use get_pandas_estimated_parameters(estimation_results=my_results) instead
  Swiss5.get_estimated_parameters()


,Name,Value,Robust std err.,Robust t-stat.,Robust p-value
0,asc_route1,-0.016562,0.033936,-0.488027,0.625531
1,b_cost,-0.530470,0.125845,-4.215267,0.000025


## MODEL 6

In [53]:
# Define beta values to be estimated
## Beta('name', initial value, lower bound, upper bound, reference)
## Reference is used to set the reference alternative in a logit model (1 = yes, 0 = no)
asc_route1 = Beta('asc_route1', 0, None, None, 0)
asc_route2 = Beta('asc_route2', 0, None, None, 1) # reference alternative
b_cost = Beta('b_cost', 0, None, None, 0)
b_hw = Beta('b_hw', 0, None, None, 0)

# Utility functions
V_route1 = asc_route1 + b_cost * tc1 + b_hw * hw1
V_route2 = asc_route2 + b_cost * tc2 + b_hw * hw2

# Dictionary defining alternatives mapping
V = {1: V_route1, 2: V_route2}

# Dictionary defining availability mapping
av = {1: 1, 2: 1} # Both alternatives are ALWAYS available

# Define the choice model
# loglogit(Alternatives mapping, Availability mapping, Choice variable)
logprob= models.loglogit(V, av, choice)

# Estimate Model
#the_biogeme = bio.BIOGEME(database, logprob)
the_biogeme = bio.BIOGEME(database, logprob)
the_biogeme.model_name = 'Swiss6' # Set the model name

#Calculate null Loglikelihood
the_biogeme.calculate_null_loglikelihood(av)

# Save the estimation results
Swiss6 = the_biogeme.estimate()

#Print the results
print(Swiss6.short_summary())
Swiss6.get_estimated_parameters()

Results for model Swiss6
Nbr of parameters:		3
Sample size:			3492
Excluded data:			0
Null log likelihood:		-2420.47
Final log likelihood:		-2232.557
Likelihood ratio test (null):		375.8266
Rho square (null):			0.0776
Rho bar square (null):			0.0764
Akaike Information Criterion:	4471.113
Bayesian Information Criterion:	4489.588



/var/folders/g0/rncnm3l57mvd8n3r12bv0_br0000gn/T/ipykernel_2916/1034966405.py:36: DeprecationWarning: get_estimated_parameters is deprecated. Use get_pandas_estimated_parameters(estimation_results=my_results) instead
  Swiss6.get_estimated_parameters()


,Name,Value,Robust std err.,Robust t-stat.,Robust p-value
0,asc_route1,-0.012333,0.035731,-0.345169,0.729967
1,b_cost,-0.001915,0.006143,-0.311730,0.755246
2,b_hw,-0.025917,0.001443,-17.959741,0.000000


## Ben-Akiva and Swait Test

$$BST = Pr\left[\left(\bar{\rho}^2_{\text{HI}}-\bar{\rho}^2_{\text{LO}}\right)\geq z \right] \leq \Phi \left[-\sqrt{-2z\mathcal{L}\left(0\right) + \left( \kappa_{\text{HI}} - \kappa_{\text{LO}} \right) }  \right]$$

In [54]:
def BST_Test(model_1,model_2):
    L0 = float(model_1.get_general_statistics().get('Null log likelihood') )
    model_hi = model_1 if model_1.rho_bar_square_null > model_2.rho_bar_square_null else model_2
    model_lo = model_2 if model_1.rho_bar_square_null > model_2.rho_bar_square_null else model_1
    #compute BST value and p-value
    bst_value = -1*math.sqrt(-2*(model_hi.final_loglikelihood - model_lo.final_loglikelihood)*L0 + (model_hi.number_of_parameters - model_lo.number_of_parameters))
    p_value = stats.norm.cdf(bst_value)
    result = lambda x: "Reject the null hypothesis: The " + model_2.model_name + " provides a significantly better fit." if p_value < 0.05 else "Fail to reject the null hypothesis: The " + model_1.model_name + " is sufficient."
    return {
        "BST_value": bst_value,
        "p_value": float(p_value),
        "Reject_null": result(p_value)
    }

BST_Test(Swiss1, Swiss5)


{'BST_value': -548.9022603295497,
 'p_value': 0.0,
 'Reject_null': 'Reject the null hypothesis: The Swiss5 provides a significantly better fit.'}